In [1]:
# The very first step is the installation of Pandas and dotenv libraries:
# We can do this in two ways:
# 1) Via PIP using pycharm terminal (or directly through command line) 
    # - pycharm terminal commands:
        # pip install pandas
        # pip install python-dotenv
        # pip install google-api-python-client
# 2) Installing the packages via Anaconda Navigator (this is how I did it)
# Just install python-dotenv and pandas in the Anaconda Environments, and then install google-api-client directly in the jupyter lab

In [2]:
# jupyter lab command to install google-api-client
!pip install google-api-python-client

In [3]:
# import os, pandas, dotenv and google-api-client
# - 'os' module to interact with the operating system, 
# - Pandas library for working with data
# - load_dotenv' from dotenv to load environment variables
# - 'build' from googleapiclient to create and manage Google API services
import os
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build

In [4]:
# Load environment variables from the specified .env file
load_dotenv(dotenv_path=".env", override=True)

True

In [5]:
# Retrieve the YouTube API key from environment variables and set the version of the YouTube API to use (v3)
API_KEY = os.getenv("YOUTUBE_API_KEY")
API_VERSION = 'v3'

In [6]:
# Set a YouTube API service object using the API key and version
youtube = build('youtube', API_VERSION, developerKey=API_KEY)

In [7]:
# Function to retrieve channel statistics (name, subscribers, views, videos) from YouTube API
def get_channel_stats(youtube, channel_id):
    # Send a request to the YouTube API to get channel details and statistics
        # Request the 'snippet' (channel info) and 'statistics' parts
        # Use the provided channel ID
    request = youtube.channels().list(
        part='snippet, statistics',
        id=channel_id
    )
    # Execute the request to get the response
    response = request.execute()
    # Check if the channel data exists in the response
    if response['items']:
        # Extract channel data and storing it in a dictionary
        data = dict(channel_name=response['items'][0]['snippet']['title'],
                    total_subscribers=response['items'][0]['statistics']['subscriberCount'],
                    total_views=response['items'][0]['statistics']['viewCount'],
                    total_videos=response['items'][0]['statistics']['videoCount'],
        )
        # Return the extracted data
        return data
    else:
        # If no channel data is found, return None
        return None


In [8]:
# Example of channel statistics:
# first try - Puzzle guy channel
# channel_id = "UCGl7qkMPao3EBkxIDlPal4g"
channel_id = "UCGl7qkMPao3EBkxIDlPal4g"
get_channel_stats(youtube, channel_id)

{'channel_name': 'Puzzle guy',
 'total_subscribers': '2560000',
 'total_views': '1606976336',
 'total_videos': '912'}

In [9]:
# Load data from a CSV file containing information about Czech YouTube channels
youtube_df = pd.read_csv("source_czech_rep_youtube.csv")

In [10]:
# Check the dataframe content
youtube_df

,#,NAME,FOLLOWERS,ER,COUNTRY,TOPIC OF INFLUENCE,POTENTIAL REACH
0,1,Puzzle guy@UCGl7qkMPao3EBkxIDlPal4g,1.7M,0.30%,Czech Republic,NaN,501K
1,2,MenT@UCk-1Oa5qkBUXNKYdK95os8g,1.4M,0.40%,Czech Republic,NaN,432K
2,3,Игорь Линк@UCptRK95GEDXvJGOQIFg50fg,1.2M,-,Czech Republic,NaN,366K
3,4,HONEST GUIDE@UCt7oj318jVQi7vRbc1bNjJA,1.2M,0.40%,Czech Republic,NaN,348K
4,5,Jirka Král@UCCAHP3LeWVBvrMnV-Lu4FOQ,1.1M,3.20%,Czech Republic,NaN,333K
...,...,...,...,...,...,...,...
95,97,Алексей Ворон@UC1yf6M6UBEdbs-NiGZHG7QA,280K,0.20%,Czech Republic,NaN,84K
96,98,SýrYakari@UCeT4d9P-M4O35E8Lzv38LCw,280K,-,Czech Republic,NaN,84K
97,99,Lime Apple Kids@UC1OzBri7GVwuoi2GK79gDpw,274K,-,Czech Republic,NaN,82.2K
98,100,JenTheGayLord@UCbzKqKrsAK71dwHuSqCgwQQ,272K,-,Czech Republic,NaN,81.6K


In [11]:
# Extract channel IDs and removing potential duplicates
channel_ids = youtube_df['NAME'].str.split('@').str[1].unique()

In [12]:
# Check the channel id's
channel_ids[0]

'UCGl7qkMPao3EBkxIDlPal4g'

In [13]:
# Initialize a list to keep track of channel statistics
channel_data = []

In [14]:
# Loop over the channel IDs and get stats for each of them
for channel_id in channel_ids:
    stats = get_channel_stats(youtube, channel_id)
    if stats is not None:
        channel_data.append(stats)

In [15]:
# Check the final statistics 
channel_data[0]

{'channel_name': 'Puzzle guy',
 'total_subscribers': '2560000',
 'total_views': '1606976336',
 'total_videos': '912'}

In [16]:
# Convert the list of statistics to a dataframe
channel_data_df = pd.DataFrame(channel_data)

In [17]:
# Check received data in new dataframe
channel_data_df

,channel_name,total_subscribers,total_views,total_videos
0,Puzzle guy,2560000,1606976336,912
1,MenT,1540000,909023017,3042
2,Игорь Линк,1250000,244042815,466
3,HONEST GUIDE,1390000,166338940,234
4,Jirka Král,1110000,374773694,1167
...,...,...,...,...
95,Алексей Ворон,414000,72464578,4968
96,SýrYakari,354000,90936642,212
97,HORIZONSPACE,258000,14360,71
98,JenTheGayLord,263000,145037614,165


In [18]:
# Reset of dataframes index
youtube_df.reset_index(drop=True, inplace=True)
channel_data_df.reset_index(drop=True, inplace=True)

In [19]:
# Concatenate the dataframes horizontally
merged_df = pd.concat([youtube_df, channel_data_df], axis=1)

In [20]:
# Check the final dataframe
merged_df

,#,NAME,FOLLOWERS,ER,COUNTRY,TOPIC OF INFLUENCE,POTENTIAL REACH,channel_name,total_subscribers,total_views,total_videos
0,1,Puzzle guy@UCGl7qkMPao3EBkxIDlPal4g,1.7M,0.30%,Czech Republic,NaN,501K,Puzzle guy,2560000,1606976336,912
1,2,MenT@UCk-1Oa5qkBUXNKYdK95os8g,1.4M,0.40%,Czech Republic,NaN,432K,MenT,1540000,909023017,3042
2,3,Игорь Линк@UCptRK95GEDXvJGOQIFg50fg,1.2M,-,Czech Republic,NaN,366K,Игорь Линк,1250000,244042815,466
3,4,HONEST GUIDE@UCt7oj318jVQi7vRbc1bNjJA,1.2M,0.40%,Czech Republic,NaN,348K,HONEST GUIDE,1390000,166338940,234
4,5,Jirka Král@UCCAHP3LeWVBvrMnV-Lu4FOQ,1.1M,3.20%,Czech Republic,NaN,333K,Jirka Král,1110000,374773694,1167
...,...,...,...,...,...,...,...,...,...,...,...
95,97,Алексей Ворон@UC1yf6M6UBEdbs-NiGZHG7QA,280K,0.20%,Czech Republic,NaN,84K,Алексей Ворон,414000,72464578,4968
96,98,SýrYakari@UCeT4d9P-M4O35E8Lzv38LCw,280K,-,Czech Republic,NaN,84K,SýrYakari,354000,90936642,212
97,99,Lime Apple Kids@UC1OzBri7GVwuoi2GK79gDpw,274K,-,Czech Republic,NaN,82.2K,HORIZONSPACE,258000,14360,71
98,100,JenTheGayLord@UCbzKqKrsAK71dwHuSqCgwQQ,272K,-,Czech Republic,NaN,81.6K,JenTheGayLord,263000,145037614,165


In [21]:
# Drop the 'channel_name' column from combined_df 
# (column 'NAME' already exists - or we could save it and drop the NAME column, but we are gonna do it later in sql)
merged_df.drop('channel_name', axis=1, inplace=True)

In [22]:
# Last check of the final data
merged_df.head(3)

,#,NAME,FOLLOWERS,ER,COUNTRY,TOPIC OF INFLUENCE,POTENTIAL REACH,total_subscribers,total_views,total_videos
0,1,Puzzle guy@UCGl7qkMPao3EBkxIDlPal4g,1.7M,0.30%,Czech Republic,NaN,501K,2560000,1606976336,912
1,2,MenT@UCk-1Oa5qkBUXNKYdK95os8g,1.4M,0.40%,Czech Republic,NaN,432K,1540000,909023017,3042
2,3,Игорь Линк@UCptRK95GEDXvJGOQIFg50fg,1.2M,-,Czech Republic,NaN,366K,1250000,244042815,466


In [23]:
# Save the final dataframe (merged data) 
# into a CSV file using special encoding (utf-8-sig) because of compatibily with czech and russian characters
merged_df.to_csv('updated_czech_rep_youtube.csv', index=False, encoding="utf-8-sig")